In [ ]:
from transformers import pipeline

In [ ]:

# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-de-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-de-en")



In [ ]:
# Übersetze einen deutschen Text ins Englische
# Text, den du übersetzen möchtest
german_text = "Gesamtfläche"

# Tokenisierung des deutschen Textes
input_ids = tokenizer.encode(german_text, return_tensors="pt")

# Durchführung der Übersetzung
output = model.generate(input_ids)

# Dekodiere die Ausgabe in menschenlesbaren Text
english_translation = tokenizer.decode(output[0], skip_special_tokens=True)

# Gib die Übersetzung aus
print("Deutsch:", german_text)
print("Englisch:", english_translation)

In [1]:
import pandas as pd
import numpy as np

inkar = pd.read_csv("../../data/inkar_2022.csv",sep=";",decimal=",",dtype={'Kennziffer':str,'Kennziffer_EU':str,'Name':str}, low_memory=False)
inkar.head()

,Bereich,ID,Indikator,Raumbezug,Kennziffer,Name,Zeitbezug,Wert
0,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,001,Flensburg,1995,10494.0
1,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,001,Flensburg,1996,11921.0
2,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,001,Flensburg,1997,13213.0
3,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,001,Flensburg,1998,13301.0
4,Absolutzahlen,2176,Arbeitslose,Arbeitsmarktregionen,001,Flensburg,1999,12048.0


In [ ]:
raumbezug_selection = ["Kreise","Gemeinden", "Gemeindeverbände"]
temp = inkar[inkar['Raumbezug'].isin(raumbezug_selection)]

menu = temp.groupby(['Bereich','Indikator','ID']).aggregate({'Zeitbezug':['min','max']})
menu.columns = ["_".join(a) for a in menu.columns.to_flat_index()]
menu = menu.merge(temp.pivot_table(index=['Bereich','Indikator','ID'], columns='Raumbezug',values='Kennziffer',aggfunc=np.count_nonzero),left_index=True,right_index=True)

for column in raumbezug_selection:
    menu[column] = (menu[column]>0).replace({True: 'X', False: ""})

menu = menu.assign(
    select="",
    category="",
    category_de=menu.index.get_level_values('Bereich'),
    category_nl="",
    name="",
    name_de=menu.index.get_level_values('Indikator'),
    name_nl="",
    source_de="Inkar",
    source_id_de=menu.index.get_level_values('ID'),
    source_nl="",
    source_id_nl="",
)

column_order = ['select', 'category', 'category_de', 'category_nl', 'name', 'name_de', 'name_nl', 'source_de', 'source_id_de', 'source_nl', 'source_id_nl', 'Zeitbezug_min', 'Zeitbezug_max', 'Gemeinden', 'Gemeindeverbände', 'Kreise']
menu = menu[column_order]

menu.reset_index(inplace=True, drop=True)
menu

In [ ]:
for index in menu.index:
    german_text = menu['category_de'][index]

    input_ids = tokenizer.encode(german_text, return_tensors="pt")
    output = model.generate(input_ids)
    english_translation = tokenizer.decode(output[0], skip_special_tokens=True)

    menu['category'][index] = english_translation
    print(menu['category'][index])


print(menu['category'])



In [ ]:
for index in menu.index:
    print(index)
    german_text = menu['name_de'][index]

    input_ids = tokenizer.encode(german_text, return_tensors="pt")
    output = model.generate(input_ids)
    english_translation = tokenizer.decode(output[0], skip_special_tokens=True)

    menu['name'][index] = english_translation




In [12]:
menu.head()
menu.to_excel("C:/Users/juliu/VSCODE/VSCODE/inkar-indicators/src/out/output.xlsx", index=True)
